In [1]:
import pandas as pd
import numpy as np
import pickle
import json
import re

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
# preparamos o driver do Chrome
driver = webdriver.Chrome('path/chromedriver')

<ipython-input-2-c16ba3ec765d>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('path/chromedriver')


In [3]:
# arquivo de entrada com dataset do TMDB
file = "data_output/TMDB_2021_4000.csv"

# fazemos a importacao do dataset do TMDB
df0 = pd.read_csv(file)

# Criacao do dataset com os dados do Rotten Tomatoes

In [4]:
# criamos esta funcao para que os nomes dos filmes sejam transformados nos nomes dos filmes da url
# exemplo: The Matrix ---> the_matrix, desta forma conseguimos acessar as urls

def rottenize(title):
    '''
    Esta funcao sustitui os carateres por vazio ou por _
    para depois entrar nos links (urls) dos filmes no rotten
    '''
    title = title.translate(str.maketrans({':': '', "'": '', ".":"", ",":"", 
                                          "-":"_", " ":"_"}))
    
    return title.lower()

In [5]:
# criamos uma lista vazia para armazenar as urls dos filmes do dataset do TMDB
links = []
for tit in df0['original_title'].apply(rottenize):
    links.append('https://www.rottentomatoes.com/m/{}'.format(tit))

In [6]:
# selecionamos um pedaco da lista de links de filmes comencando pelo "start"
# links = links[start:]

def create_df(n):
    '''
    n: numero de filmes a coletar
    start: numero da lista do link do filme que vamos comecar a coletar
    '''  
    # neste dataset iremos armazenar as informacoes extraidas do Rotten Tomatoes
    df = pd.DataFrame()
    
    # listas para armazenar a informacao
    nomes = []
    TOMATOMETER = []
    TOMATOMETER_rev = []
    aud_scr = []
    aud_scr_rev = []

    genre = []
    language = []
    director = []
    producer = []
    writer = []
    rel_date_th = []
    rel_date_stream = []
    box = []
    runtime = []
    distributor = []
    description = []

#     for k in range(1000):
#     #     driver.get('https://www.rottentomatoes.com/browse/movies_in_theaters/?page={}'.format(j))
#         url = 'https://www.rottentomatoes.com/browse/movies_in_theaters/?page={}'.format(k)
#     #     print(url, end="\r", flush=True)


    for j in range(len(links[:n])):
        
        # mostra quantos filmes ja varreu
        print(j, end="\r", flush=True)
        
        # entramos no site do filme
        driver.get(links[j])
        
        # se aparecer algun erro no navegador voltamos ao comeco do loop
        try:
            # informacoes da descricao do filme
            info_we = driver.find_elements(By.CLASS_NAME, "media-body")
            # informações mais breves da descricao
            short_info = info_we[0].text.split('\n')[2:]

            # informacoes principais do filme: nome, scores e quantidade de reviews
            info_main = driver.find_elements(By.CLASS_NAME, "scoreboard")
            info_main = info_main[0].text.split('\n')

            # dicionario para armazenar as infos do filme
            short_info_dic = dict()

            # acrescentamos tambem as informacoes principais
            short_info_dic['Nome'] = info_main[0]
            short_info_dic['TOMATOMETER'] = info_main[2]
            short_info_dic['TOMATOMETER reviews'] = info_main[4]
            short_info_dic['AUDIENCE SCORE'] = info_main[5]
            short_info_dic['AUDIENCE SCORE reviews'] = info_main[7]
        except:
            continue

        # informacoes breves do filme
        for i in range(len(short_info)):
            infos = short_info[i].split(': ')
            try:
                short_info_dic[infos[0]] = infos[1]
            except:
                short_info_dic[infos[0]] = np.nan

        # resumo e descricao do filme
        short_info_dic[info_we[0].text.split('\n')[0]] = info_we[0].text.split('\n')[1]

        # colocamos as informacoes nas listas
        try:
            nomes.append(short_info_dic['Nome'])
        except:
            nomes.append(np.nan)
        
        try:
            TOMATOMETER.append(short_info_dic['TOMATOMETER'])
        except:
            TOMATOMETER.append(np.nan)
        
        try:
            TOMATOMETER_rev.append(short_info_dic['TOMATOMETER reviews'])
        except:
            TOMATOMETER_rev.append(np.nan)
        
        try:
            aud_scr.append(short_info_dic['AUDIENCE SCORE'])
        except:
            aud_scr.append(np.nan)
        
        try:
            aud_scr_rev.append(short_info_dic['AUDIENCE SCORE reviews'])
        except:
            aud_scr_rev.append(np.nan)
        
        try:
            genre.append(short_info_dic['Genre'])
        except:
            genre.append(np.nan)
        
        try:
            language.append(short_info_dic['Original Language'])
        except:
            language.append(np.nan)
        
        try:
            director.append(short_info_dic['Director'])
        except:
            director.append(np.nan)
        
        try:
            producer.append(short_info_dic['Producer'])
        except:
            producer.append(np.nan)
        
        try:
            writer.append(short_info_dic['Writer'])
        except:
            writer.append(np.nan)
        
        try:
            rel_date_th.append(short_info_dic['Release Date (Theaters)'])
        except:
            rel_date_th.append(np.nan)
        
        try:
            rel_date_stream.append(short_info_dic['Release Date (Streaming)'])
        except:
            rel_date_stream.append(np.nan)
        
        try:
            box.append(short_info_dic['Box Office (Gross USA)'])
        except:
            box.append(np.nan)
        
        try:
            runtime.append(short_info_dic['Runtime'])
        except:
            runtime.append(np.nan)
        
        try:
            distributor.append(short_info_dic['Distributor'])
        except:
            distributor.append(np.nan)
            
        try:
            description.append(short_info_dic['MOVIE INFO'])
        except:
            description.append(np.nan)
        
    df = pd.DataFrame() 

    df['Movie Name'] = nomes
    df['Audience Score'] = aud_scr
    df['AUDIENCE SCORE reviews'] = aud_scr_rev
    df['Tomatometer Score'] = TOMATOMETER
    df['TOMATOMETER reviews'] = nomes
    df['Genre'] = genre
    df['Original Language'] = language
    df['Director'] = director
    df['Producer'] = producer
    df['Writer'] = writer
    df['Release Date (Theaters)'] = rel_date_th
    df['Release Date (Streaming)'] = rel_date_stream
    df['box office'] = box
    df['Runtime'] = runtime
    df['Distributor'] = distributor
    df['MOVIE INFO'] = description
        
    return df

In [7]:
# %%timeit
df = create_df(len(links))
# df = create_df(10)

In [8]:
df

,Movie Name,Audience Score,AUDIENCE SCORE reviews,Tomatometer Score,TOMATOMETER reviews,Genre,Original Language,Director,Producer,Writer,Release Date (Theaters),Release Date (Streaming),box office,Runtime,Distributor,MOVIE INFO
0,SOLILOQUY,--,0 Ratings,--,SOLILOQUY,Animation,English,Martine Syms,Rocket Caleshu,NaN,NaN,NaN,NaN,7m,NaN,"Kita, a cultural commentator, questions consci..."
1,HANDS,--,0 Ratings,--,HANDS,Drama,English,Xavi José,"Gil Gilbert, Xavi José, Carter Pilcher",Pep Garrido,NaN,"Mar 23, 2017",NaN,9m,NaN,Two friends struggle with a terrible secret th...
2,WHEN WERE YOU BORN?,--,Fewer than 50 Ratings,42%,WHEN WERE YOU BORN?,Mystery & Thriller,English,William McGann,NaN,Anthony Coldeway,NaN,NaN,NaN,1h 5m,NaN,A Chinese astrologer (Anna May Wong) helps the...
3,LAST SEEN,--,0 Ratings,--,LAST SEEN,"Drama, Mystery & Thriller",English,Eva Ilona Brzeski,NaN,Holiday Reinhorn,NaN,NaN,NaN,1h 30m,NaN,A town searches for a missing high school stud...
4,CHITRANGI,--,0 Ratings,--,CHITRANGI,Drama,Tamil,NaN,NaN,NaN,NaN,NaN,NaN,2h 27m,NaN,A young woman unknowingly marries a king.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,LOS ANGELES,--,0 Ratings,--,LOS ANGELES,Drama,English,Kishore Tummala,"Chandra Chellapilla, Kishore Tummala",Kishore Tummala,NaN,NaN,NaN,10m,NaN,A psychiatrist helps an irritated stranger in ...
744,SOOYII,--,0 Ratings,--,SOOYII,"History, Drama",English,Krisztian Kery,"Scott Duthie, Pat Judge Hall, Richard Hocutt, ...",Richard Hocutt,NaN,NaN,NaN,1h 25m,NaN,"A young Pikuni man, the lone survivor of a dea..."
745,THE FEMALE HUSTLER,--,0 Ratings,--,THE FEMALE HUSTLER,"Crime, Drama",English,Dom Campbell,NaN,Dom Campbell,NaN,NaN,NaN,2h 4m,NaN,A women faces the ultimate betrayal after star...
746,ONE KISS,38%,Fewer than 50 Ratings,--,ONE KISS,Drama,Italian,Ivan Cotroneo,Francesca Cima,Ivan Cotroneo,NaN,"Feb 7, 2017",NaN,1h 42m,NaN,Three friends from northern Italy deal with in...


In [10]:
df.to_csv("data_output/Rotten_2021_4000.csv", index=False)

In [9]:
# @timeit

df.to_csv("data_output/Rotten_2021_4000.csv", index=False)

,Movie Name,Audience Score,AUDIENCE SCORE reviews,Tomatometer Score,TOMATOMETER reviews,Genre,Original Language,Director,Producer,Writer,Release Date (Theaters),Release Date (Streaming),box office,Runtime,Distributor,MOVIE INFO
0,SOLILOQUY,--,0 Ratings,--,SOLILOQUY,Animation,English,Martine Syms,Rocket Caleshu,NaN,NaN,NaN,NaN,7m,NaN,"Kita, a cultural commentator, questions consci..."
1,HANDS,--,0 Ratings,--,HANDS,Drama,English,Xavi José,"Gil Gilbert, Xavi José, Carter Pilcher",Pep Garrido,NaN,"Mar 23, 2017",NaN,9m,NaN,Two friends struggle with a terrible secret th...
2,WHEN WERE YOU BORN?,--,Fewer than 50 Ratings,42%,WHEN WERE YOU BORN?,Mystery & Thriller,English,William McGann,NaN,Anthony Coldeway,NaN,NaN,NaN,1h 5m,NaN,A Chinese astrologer (Anna May Wong) helps the...
3,LAST SEEN,--,0 Ratings,--,LAST SEEN,"Drama, Mystery & Thriller",English,Eva Ilona Brzeski,NaN,Holiday Reinhorn,NaN,NaN,NaN,1h 30m,NaN,A town searches for a missing high school stud...
4,CHITRANGI,--,0 Ratings,--,CHITRANGI,Drama,Tamil,NaN,NaN,NaN,NaN,NaN,NaN,2h 27m,NaN,A young woman unknowingly marries a king.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,LOS ANGELES,--,0 Ratings,--,LOS ANGELES,Drama,English,Kishore Tummala,"Chandra Chellapilla, Kishore Tummala",Kishore Tummala,NaN,NaN,NaN,10m,NaN,A psychiatrist helps an irritated stranger in ...
744,SOOYII,--,0 Ratings,--,SOOYII,"History, Drama",English,Krisztian Kery,"Scott Duthie, Pat Judge Hall, Richard Hocutt, ...",Richard Hocutt,NaN,NaN,NaN,1h 25m,NaN,"A young Pikuni man, the lone survivor of a dea..."
745,THE FEMALE HUSTLER,--,0 Ratings,--,THE FEMALE HUSTLER,"Crime, Drama",English,Dom Campbell,NaN,Dom Campbell,NaN,NaN,NaN,2h 4m,NaN,A women faces the ultimate betrayal after star...
746,ONE KISS,38%,Fewer than 50 Ratings,--,ONE KISS,Drama,Italian,Ivan Cotroneo,Francesca Cima,Ivan Cotroneo,NaN,"Feb 7, 2017",NaN,1h 42m,NaN,Three friends from northern Italy deal with in...
